# sbPoynETNRPy: An Einstein Toolkit Thorn for Computing the 4-Velocity Time-Component $u^0$, the Magnetic Field Measured by a Comoving Observer $b^{\mu}$, and the Poynting Vector $S^i$

<font color='red'>**This module has not yet been validated against a trusted code.**</font>

In the [previous tutorial module](Tutorial-u0_smallb_Poynting-Cartesian.ipynb), we constructed within SymPy full expressions for the 4-velocity time-component $u^0$, the magnetic field (measured by a comoving observer) $b^{\mu}$, and the Poynting vector $S^i$.

Here we will work through the steps necessary to construct an Einstein Toolkit diagnostic thorn (module) that uses ADMBase and HydroBase variables as input into the NRPy+-generated SymPy expressions for $b^{\mu}$, $b^2$, and the Poynting Vector $S^i$, outputting to gridfunctions $\text{smallb4U[]}$, $\text{smallb2}$, and $\text{SPoyn[]}$, respectively. This tutorial is in two steps:

1. Call on NRPy+ to convert the SymPy expressions for $b^{\mu}$, $b^2$, and the Poynting Vector $S^i$ to C code kernels
1. Build up the needed Einstein Toolkit infrastructure to implement the kernels
    1. The C code calling routines, and
    1. The .ccl files

## Step 1: Call on NRPy+ to convert the SymPy expressions for $b^{\mu}$, $b^2$, and the Poynting Vector $S^i$ to C code kernels

In [1]:
# Step 1a: import all needed modules from NRPy+:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
from outputC import *
import sympy as sp

In [2]:
# Step 1b: Initialize  parameters (stub; there are none for this module)
thismodule = __name__

We will to disable verbose output in the NRPy+ outputC function. This is an important step in this case, because our final expressions are very large. Verbose output, when enabled, will print (in comments) the input SymPy expressions to the top of the file *without* CSE, resulting here in an *enormous* output file.

We will also declare the additional gridfunctions we need for this thorn:

**Inputs from ADMBase:**
* the physical metric $\gamma_{ij}$
* the spacetime gauge quantities $\alpha$ and $\beta^i$

**Inputs from HydroBase:**
* the Valencia 3-velocity $v^i_{(n)}$
* the densitized magnetic field of a normal observer $\tilde{B}^i$

**Output gridfunctions:**
* the magnetic field as observed in a frame comoving with the plasma $b^\mu$ ($\text{smallb4U[]}$)
* twice the magnetic pressure $2 P_{\rm mag} = b_\mu b^\mu = b^2$ ($\text{smallb2}$)
* the Poynting vector $S^i$ ($\text{SPoyn[]}$)

In [3]:
# Step 1c: Force outputC::outCverbose to be false for this module to avoid gigantic C files
#          filled with the non-CSE expressions for the Weyl scalars.
par.set_parval_from_str("outputC::outCverbose",False) # To prevent absurdly large output files.
# Step 1d: Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Step 1e: declare the additional gridfunctions (i.e., functions whose values are declared 
#          at every grid point, either inside or outside of our SymPy expressions) needed
#         for this thorn

# INPUT GRIDFUNCTIONS:
gammaDD = ixp.register_gridfunctions_for_single_rank2("AUX","gammaDD", "sym01") # The AUX or EVOL designation is *not*
                                                                                # used in diagnostic modules.
betaU = ixp.register_gridfunctions_for_single_rank1("AUX","betaU") # The AUX or EVOL designation is *not*
                                                                   # used in diagnostic modules.
alpha = gri.register_gridfunctions("AUX","alpha") # The AUX or EVOL designation is *not*
                                                  # used in diagnostic modules.
ValenciavU = ixp.register_gridfunctions_for_single_rank1("AUX","ValenciavU") # The AUX or EVOL designation is *not*
                                                                             # used in diagnostic modules.
BtildeU = ixp.register_gridfunctions_for_single_rank1("AUX","BtildeU") # The AUX or EVOL designation is *not*
                                                                       # used in diagnostic modules.

# OUTPUT GRIDFUNCTIONS:
smallb4U = ixp.register_gridfunctions_for_single_rank1("AUX","smallb4U",DIM=4) # The AUX or EVOL designation is *not*
                                                                               # used in diagnostic modules.
smallb2 = gri.register_gridfunctions("AUX","smallb2") # The AUX or EVOL designation is *not*
                                                    # used in diagnostic modules.
PoynSU = ixp.register_gridfunctions_for_single_rank1("AUX","PoynSU") # The AUX or EVOL designation is *not*
                                                                     # used in diagnostic modules.

# Step 1f: Call the NRPy+ module to set up the SymPy expressions for the output, as well as the C code for computing u^0
import u0_smallb_Poynting__Cartesian.u0_smallb_Poynting__Cartesian as u0etc
u0etc.compute_u0_smallb_Poynting__Cartesian(gammaDD,betaU,alpha,ValenciavU,BtildeU)

# Step 1g: Set the gridfunction memory access type to "ETK":
par.set_parval_from_str("GridFuncMemAccess","ETK")

# Step 1h: Make output directories:
!mkdir sbPoynETNRPy     2>/dev/null # 2>/dev/null: Don't throw an error or warning if the directory already exists.
!mkdir sbPoynETNRPy/src 2>/dev/null # 2>/dev/null: Don't throw an error or warning if the directory already exists.

# Step 1i: Output routine for computing u0:
with open("sbPoynETNRPy/src/u0.h", "w") as file:
    file.write(str(u0etc.computeu0_Cfunction))
    print("Wrote to file \""+file.name+"\"")

# Step 1j: Use NRPy+'s outputC to convert the SymPy expressions for smallb4U, smallb2, and PoynSU to C code:
outputC([u0etc.smallb4U[0],u0etc.smallb4U[1],u0etc.smallb4U[2],u0etc.smallb4U[3],u0etc.smallb2, 
         u0etc.PoynSU[0],u0etc.PoynSU[1],u0etc.PoynSU[2]],
         [gri.gfaccess("","smallb4U0"),gri.gfaccess("","smallb4U1"),gri.gfaccess("","smallb4U2"),gri.gfaccess("","smallb4U3"),
          gri.gfaccess("","smallb2"),
          gri.gfaccess("","PoynSU0"),gri.gfaccess("","PoynSU1"),gri.gfaccess("","PoynSU2")],
       filename="sbPoynETNRPy/src/smallb4U_smallb2_PoynSU.h")

Wrote to file "sbPoynETNRPy/src/u0.h"
Wrote to file "sbPoynETNRPy/src/smallb4U_smallb2_PoynSU.h"


## Step 2: Build up the needed Einstein Toolkit infrastructure to implement the NRPy+-generated C code kernels

### Step 2a: We next need to write the C code functions called by the Einstein Toolkit scheduler that incorporate the above ".h" files:

In [4]:
%%writefile sbPoynETNRPy/src/sbPoynETNRPy.c

#include <math.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

void sbPoynETNRPy_lowlevel(const cGH* restrict const cctkGH,const int *cctk_lsh,
                           const CCTK_REAL *gammaDD00GF,const CCTK_REAL *gammaDD01GF,const CCTK_REAL *gammaDD02GF,
                           const CCTK_REAL *gammaDD11GF,const CCTK_REAL *gammaDD12GF,const CCTK_REAL *gammaDD22GF,
                           const CCTK_REAL *alphaGF,
                           const CCTK_REAL *betaU0GF,const CCTK_REAL *betaU1GF,const CCTK_REAL *betaU2GF,
                           const CCTK_REAL *ValenciavU0GF,const CCTK_REAL *ValenciavU1GF,const CCTK_REAL *ValenciavU2GF,
                           const CCTK_REAL *BtildeU0GF,const CCTK_REAL *BtildeU1GF,const CCTK_REAL *BtildeU2GF,

                           CCTK_REAL *smallb4U0GF,CCTK_REAL *smallb4U1GF,CCTK_REAL *smallb4U2GF,CCTK_REAL *smallb4U3GF,
                           CCTK_REAL *smallb2GF,
                           CCTK_REAL *PoynSU0GF,CCTK_REAL *PoynSU1GF,CCTK_REAL *PoynSU2GF) {

#pragma omp parallel for
    for(int i2=0;i2<cctk_lsh[2];i2++) for(int i1=0;i1<cctk_lsh[1];i1++) for(int i0=0;i0<cctk_lsh[0];i0++) {
        const CCTK_REAL gammaDD00 = gammaDD00GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];
        const CCTK_REAL gammaDD01 = gammaDD01GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];
        const CCTK_REAL gammaDD02 = gammaDD02GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];
        const CCTK_REAL gammaDD11 = gammaDD11GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];
        const CCTK_REAL gammaDD12 = gammaDD12GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];
        const CCTK_REAL gammaDD22 = gammaDD22GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];

        const CCTK_REAL alpha = alphaGF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];

        const CCTK_REAL betaU0 = betaU0GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];
        const CCTK_REAL betaU1 = betaU1GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];
        const CCTK_REAL betaU2 = betaU2GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];
        
        // Valencia 3-velocity may be ceiling
        CCTK_REAL ValenciavU0 = ValenciavU0GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];
        CCTK_REAL ValenciavU1 = ValenciavU1GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];
        CCTK_REAL ValenciavU2 = ValenciavU2GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];

        const CCTK_REAL BtildeU0 = BtildeU0GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];
        const CCTK_REAL BtildeU1 = BtildeU1GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];
        const CCTK_REAL BtildeU2 = BtildeU2GF[CCTK_GFINDEX3D(cctkGH, i0,i1,i2)];

        CCTK_REAL u0;
#include "u0.h"
#include "smallb4U_smallb2_PoynSU.h""
    }
}

Overwriting sbPoynETNRPy/src/sbPoynETNRPy.c
